In [1]:
# ==============================================================================
#  CORE LIBRARIES & INITIAL SETUP
# ==============================================================================

# --- General Purpose Imports ---
import os
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split

# --- PyTorch ---
# Consolidating torch import for deep learning operations.
import torch

# --- spaCy ---
# Load the German spaCy model for NLP tasks like tokenization and POS tagging.
# This 'nlp' object will be used globally in the notebook.
import spacy
nlp = spacy.load("de_core_news_lg")


# ==============================================================================
#  HUGGING FACE LIBRARY IMPORTS
# ==============================================================================

# --- Core Transformers ---
# Import main classes for models, tokenizers, and the training pipeline.
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    GenerationConfig
)

# --- TRL (Transformer Reinforcement Learning) ---
# Import specific classes for PPO training and value-head models.
from trl import PPOConfig, PPOTrainer, AutoModelForCausalLMWithValueHead

# --- Datasets ---
# Import classes for handling and manipulating datasets efficiently.
from datasets import Dataset, DatasetDict


# ==============================================================================
#  ENVIRONMENT & LOGGING CONFIGURATION
# ==============================================================================

# --- Set Log Verbosity ---
# Reduce the log output from the transformers library to keep the console clean.
# This will only show warnings and errors, hiding routine progress info.
import transformers
transformers.logging.set_verbosity_warning()

# --- Disable Tokenizer Parallelism ---
# Set an environment variable to prevent a deadlock warning/error when using
# `datasets.map()` with multiprocessing. The `datasets` library will handle
# the parallelism, so the tokenizer's internal parallelism is disabled.
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# --- Configure Hardware Device (CPU vs. Apple Silicon GPU) ---
# Check for the availability of Apple's Metal Performance Shaders (MPS) for GPU
# acceleration and set the global device accordingly.
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

W0925 17:34:31.716000 5276 site-packages/torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


'NoneType' object has no attribute 'cadam32bit_grad_fp32'
Using device: mps


/opt/miniconda3/envs/trl-env/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [2]:
# This special import uses the 'ipynb' library to load functions directly
# from the `reward_computation.ipynb` file. This helps keep the project organized
# by separating the complex reward logic from the main training pipeline.
from ipynb.fs.full.reward_computation import (
    # The main function that combines all sub-scores into a final weighted reward.
    compute_reward, 
    # Calculates the score based on adherence to simplification rules (e.g., no compounds).
    rule_compliance_score, 
    # Calculates the score for grammatical correctness using language_tool_python.
    calculate_grammar_score,
    # Calculates the meaning preservation score using the SBERT model.
    calculate_semantic_similarity
)

Loading data file - german_dict/german_utf8.dic
1.0
Loading BERT model...


No sentence-transformers model found with name deepset/gbert-large. Creating a new one with mean pooling.
/opt/miniconda3/envs/trl-env/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/83.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Model similarity function set to: 'dot'
Model loaded.


## Choice for the  RewardModel

AutoModelForSequenceClassification
    - a family of AutoModel classes by huggingface
  - > loads a pretrained encoder and attach a classification head on top
That classification head is
 - for classification: outputs logits over N classes (e.g. positive/negative)
 - for regression: if I set num_labels=1, it outputs a single scalar

### BERT base vs DistilBERT

BERT base uncased
 - higher accuracy
 - 12 layers, 110M parameters.
 - High accuracy, but slower and heavier to train/infer.

DistilBERT - _Picking This one_
- faster PPO training
- A distilled (compressed) version of BERT.
- ~40% smaller, ~60% faster, only ~3% loss in accuracy.
- Often preferred as a reward model because PPO will call it a LOT (every generation gets scored).
- Faster inference = cheaper RL training.

In [3]:
# Update your model_init function: Modify your model initialization to move the model to the MPS device right after it's loaded.
def model_init():
    model_name = "distilbert-base-german-cased"
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name, num_labels=1, problem_type="regression"
    )
    return model.to(device) 

## Hyperparameter


### How Optuna Smartly Finds Hyperparameters

Optuna doesn't use a simple grid or random search. By default, it uses a smart Bayesian optimization algorithm called **TPE (Tree-structured Parzen Estimator)** that learns from past results to make better guesses over time.

#### The TPE Process
1.  **Explore:** It starts with a few random trials to gather initial data.
2.  **Model:** It divides the results into a "good" group (e.g., top 25% of scores) and a "bad" group.
3.  **Suggest:** For the next trial, it suggests a new set of hyperparameters that are statistically more likely to be in the "good" group than the "bad" group.

#### Analogy: Finding the Best Setting 💡
* **Grid Search:** Checks every single possible setting. Exhaustive but very slow.
* **Random Search:** Tries random settings. Better than grid search, but doesn't learn from its mistakes.
* **Optuna (TPE):** Plays "Hot and Cold." It uses the feedback from previous trials ("warmer" or "colder") to make its next guess much more intelligent, focusing its search on the most promising areas of the hyperparameter space.

In [4]:
# ==============================================================================
#  HYPERPARAMETER SEARCH CONFIGURATION
# ==============================================================================

# Define the number of different hyperparameter combinations for Optuna to try.
N_TRIALS = 5

# Define the search space for each hyperparameter in a centralized dictionary.
HP_SEARCH_SPACE = {
    "learning_rate": {
        "type": "float",
        "low": 1e-5,
        "high": 5e-5,
        "log": True,
        # What it is: Controls how much the model's weights are updated during training.
        # Why it's important: It's the most critical hyperparameter. Too high, and the model
        # won't converge; too low, and training will be extremely slow. A log scale
        # is used because the optimal value can vary by orders of magnitude.
    },
    "num_train_epochs": {
        "type": "int",
        "low": 6,
        "high": 15,
        # What it is: The total number of times the training algorithm will pass
        # through the entire training dataset.
        # Why it's important: Balances underfitting and overfitting. Too few epochs,
        # and the model won't learn enough; too many, and it may memorize the
        # training data and perform poorly on new data.
    },
    "per_device_train_batch_size": {
        "type": "categorical",
        "choices": [32, 64],
        # What it is: The number of training examples used in a single forward/backward pass.
        # Why it's important: Affects training stability and memory usage. Larger
        # batches provide more stable gradient estimates but require more GPU RAM.
    },
    "weight_decay": {
        "type": "float",
        "low": 0.0,
        "high": 0.1,
        # What it is: A regularization technique that penalizes large weights in the model.
        # Why it's important: Helps prevent overfitting by keeping the model's weights small
        # and simple, improving its ability to generalize to new data.
    },
    "lr_scheduler_type": {
        "type": "categorical",
        "choices": ["linear", "cosine"],
        # What it is: The strategy for changing the learning rate during training.
        # Why it's important: A good schedule (like a linear decay) helps the model
        # converge faster and more reliably than a constant learning rate.
    },
    "warmup_ratio": {
        "type": "float",
        "low": 0.0,
        "high": 0.1,
        # What it is: The fraction of total training steps used for a "warmup" phase.
        # Why it's important: The learning rate starts at 0 and slowly increases to its
        # target value. This prevents large, unstable updates at the beginning of
        # training, allowing the model to stabilize first.
    },
    "gradient_accumulation_steps": {
        "type": "categorical",
        "choices": [1, 2],
        # What it is: The number of smaller batches to process before performing a
        # single model weight update.
        # Why it's important: It's a trick to simulate a larger batch size without
        # using more memory. An effective batch size of 32 (16 * 2) can lead
        # to more stable training.
    }
}

In [5]:

"""
def model_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 3e-5, log=True), # typical 1e-5 to 3e-5 WHAT TO ADD
        "num_train_epochs": trial.suggest_int("num_train_epochs", 10, 10), # Kept it small for speed, HOW TO SET??
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [16, 16]),
        "weight_decay": trial.suggest_float("weight_decay", 0.0, 0.0),
    }
"""

def model_hp_space(trial):
    """
    Dynamically creates the hyperparameter search space for an Optuna trial by
    reading from the global HP_SEARCH_SPACE configuration dictionary.
    """
    params = {}
    # Loop through each parameter defined in the configuration.
    for name, config in HP_SEARCH_SPACE.items():
        param_type = config["type"]
        
        # Call the appropriate Optuna `suggest` method based on the parameter's type.
        if param_type == "float":
            params[name] = trial.suggest_float(
                name, config["low"], config["high"], log=config.get("log", False)
            )
        elif param_type == "int":
            params[name] = trial.suggest_int(
                name, config["low"], config["high"]
            )
        elif param_type == "categorical":
            params[name] = trial.suggest_categorical(
                name, config["choices"]
            )
            
    return params

## CANDIDATE_WEIGHTS

In [6]:
CANDIDATE_WEIGHTS = [
    {"name": "balanced", "weights": {"rules_score": 0.5, "meaning_score": 0.25, "grammar_score": 0.25}}, #baseline
    {"name": "rules_heavy", "weights": {"rules_score": 0.7, "meaning_score": 0.2, "grammar_score": 0.1}},
    {"name": "meaning_heavy", "weights": {"rules_score": 0.2, "meaning_score": 0.7, "grammar_score": 0.1}},
    {"name": "grammar_focused", "weights": {"rules_score": 0.2, "meaning_score": 0.1, "grammar_score": 0.7}},
]

## Data Load & Preprocessing

In [7]:
# Predefine the preprocessing function
def preprocess_function(examples, tokenizer):
    """Tokenizes the 'simplified' text column."""
    # Important: Tokenization is applied on the 'simplified' column now, which becomes the 'text' for the RM
    return tokenizer(examples["simplified"], truncation=True, padding="max_length", max_length=128)

In [8]:
# --- 1. Load, Split, and Tokenize Data ONCE ---
print("--- Loading and tokenizing data once ---")
df = pd.read_csv("data/ordered_simplifications_with_rules.csv", index_col=0)
df.info()

--- Loading and tokenizing data once ---
<class 'pandas.core.frame.DataFrame'>
Index: 11107 entries, 1 to 16546
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   original_sentence     11107 non-null  object
 1   final_simplification  11107 non-null  object
 2   applied_rules         11107 non-null  object
dtypes: object(3)
memory usage: 347.1+ KB


In [9]:
df.head()

,original_sentence,final_simplification,applied_rules
uid,,,
1,Der Iran wird teilweise aus dem Atom-Abkommen ...,Der Iran wird teilweise aus dem Atom-Abkommen ...,['normalize_verb_tense']
2,Brüssel Ursula von der Leyen ist die Präsi·Den...,Brüssel Ursula von der Leyen ist die Präsi·Den...,['split_compound']
3,Am Mittwoch hat sie ihre 1. Rede zur Lage der ...,Am Mittwoch hat sie ihre 1. Rede zur Lage der ...,['convert_word_to_number']
4,Bis zum Jahr 2030 soll es in der Europäische U...,Bis zum Jahr 2030 soll es in der Europäische U...,['normalize_verb_tense']
5,"Das ist sehr viel, denn in den letzten 29 Jahr...","Das hat ist sehr viel, denn in den letzten 29 ...",['normalize_verb_tense']


In [10]:
df.rename(columns={"original_sentence": "original", "final_simplification": "simplified"}, inplace=True)

In [11]:
#use for Smaller Dataset
#df = df.sample(n=50, random_state=42) #reduce size for testing

In [12]:
# Create the initial dataset from the full DataFrame
full_dataset = Dataset.from_pandas(df)
# Split it into train and test sets
split_dataset = full_dataset.train_test_split(test_size=0.15, seed=42)

In [13]:
# Load the tokenizer and apply tokenization on both split sets
tokenizer_rm = AutoTokenizer.from_pretrained("distilbert-base-german-cased")

# Tokenize the base train and test sets without the labels
# We keep the original text columns to calculate rewards later
tokenized_train_base = split_dataset["train"].map(
    lambda examples: preprocess_function(examples, tokenizer_rm),
    batched=True
)
tokenized_test_base = split_dataset["test"].map(
    lambda examples: preprocess_function(examples, tokenizer_rm),
    batched=True
)
print("--- Data tokenized successfully. Starting grid search... ---")

Map:   0%|          | 0/9440 [00:00<?, ? examples/s]

Map:   0%|          | 0/1667 [00:00<?, ? examples/s]

--- Data tokenized successfully. Starting grid search... ---


## Eval Custom Metrics for regression RM
- for the chosen regression RM model MSE loss is chosen
- the following metrics are also loggeed
  - MSE (Mean Squared Error) → matches your training loss, so you can track consistency.
  - MAE (Mean Absolute Error) → more interpretable (average absolute difference).
  - R² (Coefficient of Determination) → tells you how well your model explains variance (1 = perfect, 0 = baseline).

In [14]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.squeeze()   # shape: (batch,)
    labels = labels.squeeze()

    mse = mean_squared_error(labels, preds)
    mae = mean_absolute_error(labels, preds)
    r2  = r2_score(labels, preds)

    return {"mse": mse, "mae": mae, "r2": r2}

## Redirecting Standard Output 

In [15]:
"""
rrect: the print statements are coming directly from the german-compound-splitter library itself, which makes them impossible to remove by editing your own code.

This is a common issue with older or research-focused libraries. The definitive way to solve this is to temporarily redirect all output from the subprocesses to a "black hole" so the print statements are silenced.

The Solution: Redirecting Standard Output
We will create a small helper class that redirects all system output (like print) to /dev/null (an operating system-level trash can) just for the duration of the .map() call. This is an advanced but very effective technique.

Step 1: Add the Helper Class
In a new cell in your supervised_fine_tuningSFT.ipynb notebook, before your main training loop, copy and paste this helper class.
Step 2: Update Your .map() Call
Now, in your main training loop, you will wrap your .map() call inside this new context manager.

"""

import os
import sys
from contextlib import contextmanager

@contextmanager
def suppress_stdout_stderr():
    """A context manager that redirects stdout and stderr to devnull"""
    # This works on Mac/Linux to silence all output, including from underlying libraries
    with open(os.devnull, 'w') as fnull:
        old_stdout, old_stderr = sys.stdout, sys.stderr
        sys.stdout, sys.stderr = fnull, fnull
        try:
            yield
        finally:
            sys.stdout, sys.stderr = old_stdout, old_stderr

## Helper Functions for Reward Calculation

In [16]:
# ==============================================================================
#  HELPER FUNCTIONS FOR REWARD CALCULATION
# ==============================================================================

def compute_stable_parts_batched(examples):
    """
    Calculates the reward components that are stable during multiprocessing.
    
    This function processes a BATCH of text examples for high efficiency. It's
    specifically designed to be the target function for `datasets.map()` when
    running in parallel. It handles the spaCy and language_tool calculations.
    
    Args:
        examples (dict): A dictionary from `datasets.map` where each key
                         (e.g., 'simplified') maps to a list of values.
                         
    Returns:
        dict: A dictionary containing lists of the calculated scores, which
              will be added as new columns to the dataset.
    """
    # Extract the list of sentences for this batch.
    simplified_batch = examples['simplified']
    
    # Use spaCy's nlp.pipe() for highly optimized, batched NLP processing.
    # It's significantly faster than calling nlp() on each sentence individually.
    # Note: nlp.pipe() returns a generator. Since generators are consumed after one
    # iteration, we must create two separate ones for the two list comprehensions below.
    docs_for_rules = nlp.pipe(simplified_batch)
    docs_for_grammar = nlp.pipe(simplified_batch)
    
    # Calculate the scores for each document in the batch.
    rules_scores = [rule_compliance_score(doc) for doc in docs_for_rules]
    grammar_scores = [calculate_grammar_score(doc.text) for doc in docs_for_grammar]

    # The returned dictionary's keys will become the new column names in the dataset.
    return {"rules_score": rules_scores, "grammar_score": grammar_scores}


def calculate_rewards_for_split(dataset_split, weights, num_proc, desc_prefix=""):
    """
    Orchestrates the full two-stage reward calculation for a given dataset split.
    
    This function encapsulates the entire complex reward logic:
    - Stage 1: Computes stable scores (rules, grammar) in parallel for speed.
    - Stage 2: Computes the SBERT meaning score sequentially to avoid crashes.
    - Stage 3: Combines the scores from both stages using the provided weights.
    
    Args:
        dataset_split (Dataset): The dataset slice (e.g., train or test) to process.
        weights (dict): The dictionary of weights for the current grid search configuration.
        num_proc (int): The number of CPU cores to use for Stage 1.
        desc_prefix (str): A string (like "train" or "test") to label the progress bars.
        
    Returns:
        list: A list of the final, weighted reward scores for the dataset split.
    """
    # --- STAGE 1: Parallel calculation of stable scores ---
    # We silence all stdout/stderr from subprocesses to prevent the noisy
    # `german-compound-splitter` library from creating an I/O bottleneck.
    with suppress_stdout_stderr():
        stable_scores_ds = dataset_split.map(
            compute_stable_parts_batched,
            batched=True,
            batch_size=100,
            num_proc=num_proc,
            desc=f"Calculating stable scores ({desc_prefix})"
        )

    # --- STAGE 2: Sequential calculation of the SBERT meaning score ---
    # This part is run on a single core because the SentenceTransformer model
    # was found to be incompatible with multiprocessing, causing crashes.
    meaning_scores = [
        calculate_semantic_similarity(ex['original'], ex['simplified']) 
        for ex in tqdm(dataset_split, desc=f"Calculating meaning scores ({desc_prefix})")
    ]
    
    # --- STAGE 3: Combine all calculated scores ---
    # This loop combines the results from the parallel and sequential stages
    # into a final, weighted reward score for each example.
    final_rewards = []
    for i in range(len(dataset_split)):
        r_s = stable_scores_ds[i]["rules_score"]  # From Stage 1
        g_s = stable_scores_ds[i]["grammar_score"] # From Stage 1
        m_s = meaning_scores[i]                   # From Stage 2
        
        # Apply the weights for the current experiment configuration.
        reward = (weights["rules_score"] * r_s +
                  weights["grammar_score"] * g_s +
                  weights["meaning_score"] * m_s)
        final_rewards.append(reward)
        
    return final_rewards

## Helper Functions for Training Pipeline 

In [17]:

# ==============================================================================
#  HELPER FUNCTIONS FOR THE TRAINING PIPELINE
# ==============================================================================

def prepare_datasets_for_run(tokenized_train_base, tokenized_test_base, train_rewards, test_rewards):
    """Adds reward labels to the tokenized datasets and sets the final PyTorch format."""
    # Add the calculated rewards as the 'labels' column for this training run.
    train_dataset = tokenized_train_base.add_column("labels", train_rewards)
    test_dataset = tokenized_test_base.add_column("labels", test_rewards)
    
    # Set the dataset format to PyTorch tensors for compatibility with the Trainer.
    train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
    test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
    
    return train_dataset, test_dataset


def run_hyperparameter_search(trainer, config_name):
    """Runs the hyperparameter search for a given trainer and returns the best run."""
    print(f"--- Running hyperparameter search for {config_name} ---")
    
    # This call now uses the N_TRIALS variable defined in your configuration.
    best_run = trainer.hyperparameter_search(
        direction="minimize",      # We want to minimize the objective (MSE).
        hp_space=model_hp_space,   # The function that builds the search space.
        n_trials=N_TRIALS          # The number of trials, controlled from the top of the notebook.
    )
    print(f"Best run for {config_name}: {best_run}")
    return best_run


def train_and_save_final_model(trainer, best_run, output_dir_base, config_name):
    """Trains the final model using the best hyperparameters and saves it."""
    print(f"--- Training final model for {config_name} with best hyperparameters ---")
    
    # Apply the best parameters found during the search to the trainer's arguments.
    for k, v in best_run.hyperparameters.items():
        setattr(trainer.args, k, v)
    
    # Start the final training run.
    trainer.train()

    # Save the final, optimized model to a unique directory.
    final_output_dir = f"{output_dir_base}_final"
    trainer.save_model(final_output_dir)
    print(f"--- Saved final optimized model to {final_output_dir} ---\n")

"""
Reusable Summary Function:  It takes a single result dictionary and prints it in a clean format.
"""
def print_run_summary(result):
    """Prints a formatted summary for a single training run."""
    print(f"\n--- Summary for Configuration: {result['config_name']} ---")
    print("  Best Hyperparameters Found:")
    for param, value in result['best_hyperparameters'].items():
        print(f"    - {param}: {value}")
    
    print("  Final Evaluation Metrics:")
    for metric, value in result['final_metrics'].items():
        # We only print metrics that start with 'eval_' and format them nicely
        if 'eval_' in metric:
            print(f"    - {metric}: {value:.4f}")
    print("-" * 50)


## Trainig Loop with Grid Search for all 4 CANDIDATE_WEIGHTS

In [18]:
# ==============================================================================
#  MAIN GRID SEARCH AND TRAINING LOOP 
# ==============================================================================

# --- Initialize a list to store results ---
all_results = []

# This main loop iterates through each set of candidate weights, calculates the
# corresponding rewards, and then (in subsequent code) trains a reward model.
for config in CANDIDATE_WEIGHTS:
    # --- SETUP FOR THE CURRENT LOOP ITERATION ---
    config_name = config["name"]
    weights = config["weights"]
    output_dir_base = f"rm_out_{config_name}"

    print(f"\n--- Processing configuration: {config_name} ---")
    print(f"Weights: {weights}")

    # Set the number of CPU cores for parallel processing.
    num_cores_to_use = 12

    # --- REWARD CALCULATION ---
    # Call reusable orchestration function to get the rewards for both the
    # training and test sets for this specific weight configuration.
    
    print("--- Calculating rewards for the TRAINING set ---")
    train_rewards = calculate_rewards_for_split(
        split_dataset["train"], 
        weights, 
        num_cores_to_use, 
        desc_prefix="train"
    )

    print("--- Calculating rewards for the TEST set ---")
    test_rewards = calculate_rewards_for_split(
        split_dataset["test"], 
        weights, 
        num_cores_to_use, 
        desc_prefix="test"
    )
    
    print(f"--- Finished calculating rewards for {config_name} ---")


    # The 'train_rewards' and 'test_rewards' variables are now ready to be
    # used to train the reward model for this specific 'config'.

    # ==============================================================================
    #  POST-REWARD CALCULATION: DATASET PREP & MODEL TRAINING
    # ==============================================================================
    # This code block is inside the `for config in CANDIDATE_WEIGHTS:` loop.
    # The 'train_rewards' and 'test_rewards' variables are now ready.

    # --- b. Dataset Preparation ---
    # Call our helper function to create the final datasets for this specific training run.
    # This function takes the pre-tokenized text and merges it with the `train_rewards`
    # and `test_rewards` lists that were just calculated using the current `weights`.
    train_ds, test_ds = prepare_datasets_for_run(
        tokenized_train_base, tokenized_test_base, train_rewards, test_rewards
    )

    # --- c. Trainer Setup ---
    # First, we configure the training process using the `TrainingArguments` class.
    # This object holds all the settings for how the model will be trained and evaluated.
    training_args = TrainingArguments(
        output_dir=output_dir_base,      # Directory to save model checkpoints and outputs.
        use_mps_device=True,             # Explicitly leverage the Apple Silicon GPU (Metal Performance Shaders).
        evaluation_strategy="epoch",     # Run an evaluation on the test set at the end of each training epoch.
        save_strategy="epoch",           # Save a model checkpoint at the end of each epoch.
        load_best_model_at_end=True,     # After training, automatically load the checkpoint with the best performance.
        metric_for_best_model="mse",     # Use Mean Squared Error (MSE) as the key metric to determine the "best" model.
        greater_is_better=False,         # Specifies that a lower value for `metric_for_best_model` (MSE) is better.
        logging_strategy="epoch",        # How often to log training metrics. "epoch" is less noisy than "steps".,
        dataloader_pin_memory=False,     # Disable the feature and silence the warning since not applicable for Mac with an M-series chip uses a Unified Memory Architecture 
        save_total_limit=1, # Only keep the single best checkpoint on disk

    )

    # Next, we initialize the `Trainer` object.
    # This is the main orchestrator from the Hugging Face library that brings together
    # the model, datasets, and training configuration.
    trainer = Trainer(
        # `model_init` is a function that returns a fresh, untrained model.
        # This is crucial for hyperparameter search to ensure each trial starts from scratch.
        model_init=model_init,
        args=training_args,                 # The training configuration we just defined.
        train_dataset=train_ds,             # The prepared training dataset.
        eval_dataset=test_ds,               # The prepared test dataset.
        compute_metrics=compute_metrics,    # The function to calculate metrics like MSE, MAE, and R².
        tokenizer=tokenizer_rm,             # The tokenizer, used for data collation.
    )

    # --- d. Hyperparameter Search and Final Training ---
    # Now we execute the main logic using our custom helper functions for clarity.
    # First, find the best set of hyperparameters for the current reward model configuration.
    best_run = run_hyperparameter_search(trainer, config_name)
    # Then, use those best hyperparameters to train a final, optimized model.
    train_and_save_final_model(trainer, best_run, output_dir_base, config_name)

    # --- e. Evaluate the Final Model and Store Results for Summary ---
    # After the final model for this configuration has been trained, we run a final
    # evaluation on the test set to get its performance metrics.
    print(f"--- Evaluating final model for {config_name} ---")
    final_metrics = trainer.evaluate()

    # We append the results of this entire run into a list. This allows us to
    # print a full summary of all configurations after the main loop has finished.
    current_result = {
        "config_name": config_name,
        "best_hyperparameters": best_run.hyperparameters,
        "final_metrics": final_metrics
    }
    all_results.append(current_result)
    print_run_summary(current_result)


--- Processing configuration: balanced ---
Weights: {'rules_score': 0.5, 'meaning_score': 0.25, 'grammar_score': 0.25}
--- Calculating rewards for the TRAINING set ---


Calculating stable scores (train) (num_proc=12):   0%|          | 0/9440 [00:00<?, ? examples/s]

Calculating meaning scores (train):   0%|          | 0/9440 [00:00<?, ?it/s]

--- Calculating rewards for the TEST set ---


Calculating stable scores (test) (num_proc=12):   0%|          | 0/1667 [00:00<?, ? examples/s]

...

{'eval_loss': 0.000575135403778404, 'eval_mse': 0.000575135403778404, 'eval_mae': 0.016122279688715935, 'eval_r2': 0.27011221647262573, 'eval_runtime': 5.3146, 'eval_samples_per_second': 313.663, 'eval_steps_per_second': 39.325, 'epoch': 5.0}
{'loss': 0.0024, 'grad_norm': 0.13217014074325562, 'learning_rate': 0.0, 'epoch': 6.0}


  0%|          | 0/209 [00:00<?, ?it/s]

{'eval_loss': 0.0005668141529895365, 'eval_mse': 0.0005668141529895365, 'eval_mae': 0.016070080921053886, 'eval_r2': 0.28067249059677124, 'eval_runtime': 5.3092, 'eval_samples_per_second': 313.984, 'eval_steps_per_second': 39.366, 'epoch': 6.0}
{'train_runtime': 468.9679, 'train_samples_per_second': 120.776, 'train_steps_per_second': 0.947, 'train_loss': 0.03418458206159575, 'epoch': 6.0}
--- Saved final optimized model to rm_out_grammar_focused_final ---

--- Evaluating final model for grammar_focused ---


  0%|          | 0/209 [00:00<?, ?it/s]


--- Summary for Configuration: grammar_focused ---
  Best Hyperparameters Found:
    - learning_rate: 1.3053282148990605e-05
    - num_train_epochs: 6
    - per_device_train_batch_size: 64
    - weight_decay: 0.002884554211301771
    - lr_scheduler_type: linear
    - warmup_ratio: 0.08135044384600738
    - gradient_accumulation_steps: 2
  Final Evaluation Metrics:
    - eval_loss: 0.0006
    - eval_mse: 0.0006
    - eval_mae: 0.0161
    - eval_r2: 0.2807
    - eval_runtime: 5.2948
    - eval_samples_per_second: 314.8390
    - eval_steps_per_second: 39.4730
--------------------------------------------------


In [19]:
# ==============================================================================
#  FINAL SUMMARY OF ALL RUNS (NOW USING THE HELPER FUNCTION)
# ==============================================================================

print("\n\n=================================================")
print("           GRID SEARCH FINAL SUMMARY")
print("=================================================\n")

for result in all_results:
    # Call the same helper function to print each result
    print_run_summary(result)



           GRID SEARCH FINAL SUMMARY


--- Summary for Configuration: balanced ---
  Best Hyperparameters Found:
    - learning_rate: 2.836269262850202e-05
    - num_train_epochs: 7
    - per_device_train_batch_size: 64
    - weight_decay: 0.027205054637907
    - lr_scheduler_type: cosine
    - warmup_ratio: 0.05439758905893752
    - gradient_accumulation_steps: 2
  Final Evaluation Metrics:
    - eval_loss: 0.0008
    - eval_mse: 0.0008
    - eval_mae: 0.0176
    - eval_r2: 0.7519
    - eval_runtime: 5.3628
    - eval_samples_per_second: 310.8460
    - eval_steps_per_second: 38.9720
--------------------------------------------------

--- Summary for Configuration: rules_heavy ---
  Best Hyperparameters Found:
    - learning_rate: 1.4942277347418218e-05
    - num_train_epochs: 8
    - per_device_train_batch_size: 32
    - weight_decay: 0.05285585376949559
    - lr_scheduler_type: cosine
    - warmup_ratio: 0.04364105576407306
    - gradient_accumulation_steps: 2
  Final Evaluation Me

## Test Code till here 

In [20]:
# class StopExecution(Exception):
#     def _render_traceback_(self):
#         # This special method prevents the ugly traceback from appearing
#         pass

In [21]:
# raise StopExecution

In [22]:
# # Save the final, trained reward model
# trainer.save_model("model_rm")
# tokenizer_rm.save_pretrained("model_rm")

## Model_rm has been trained with the whole dataset now